In [3]:
import time
import numpy as np
from tqdm import tqdm
import scipy.stats as ss
import matplotlib.pyplot as plt


# Black and Scholes model
def d1(S0, K, r, sigma, T):
    return (np.log(S0 / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))


def d2(S0, K, r, sigma, T):
    return (np.log(S0 / K) + (r - sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))


def BlackScholes(S0, K, r, sigma, T):
    return S0 * ss.norm.cdf(d1(S0, K, r, sigma, T)) - \
           K * np.exp(-r * T) * ss.norm.cdf(d2(S0, K, r, sigma, T))


# get correlation:
def corr_c(conts, targs, conts_expt):
    cc = -np.sum((targs - np.mean(targs))*(conts - \
         conts_expt))/(np.sum((conts - conts_expt)**2))
    return cc


def heston_single(mu, rho, kappa, sigmasquare, theta, init_price,
               T, step_n, r, W, Z):
    """ single simulation of Heston model by Milstein scheme
    parameters:
        rho: correlation parameter
        sigmasquare: initial volatility
        init_price: initial price of asset
        T: time to maturity
        step_n: the steps of every simulation
        r: interest rate
        W: first Brownian Motion
        Z: second Brownian Motion
    return:
        the final asset price of this simulation
    """
    delta_t = T/step_n
    x = 0
    for i in range(0, step_n):
        delta_1 = (r-0.5*sigmasquare)*delta_t
        delta_2 = rho*np.sqrt(sigmasquare)*W[i]
        delta_3 = np.sqrt(1-rho**2)*np.sqrt(sigmasquare)*Z[i]
        x = x + delta_1 + delta_2 + delta_3
        # update sigmasquare
        sigmasquare = sigmasquare + kappa*(theta-sigmasquare)*delta_t +\
                      mu*np.sqrt(sigmasquare)*W[i] +\
                      ((mu**2)/4)*(W[i]**2-delta_t)
        if sigmasquare < 0:
            sigmasquare = -sigmasquare
    asset_final = init_price*np.exp(x)
    return asset_final


def heston(mu, rho, kappa, sigmasquare0, theta, init_price,
              sim_n, step_n, T, K, r):
    """ normal Monte Carlo Milstein simulation
    sim_n: the number of simulations
    step_n: the number of steps
    """
    payoff = np.zeros([sim_n])
    std = (T / step_n) ** 0.5
    for i in range(0, sim_n):
        W = np.random.normal(0, std, step_n + 1)
        Z = np.random.normal(0, std, step_n + 1)
        ST = heston_single(mu, rho, kappa, sigmasquare0,
             theta, init_price, T, step_n, r, W, Z)
        payoff[i] = max(ST-K, 0)
    mean = np.mean(payoff)
    variance = np.var(payoff)
    return mean, variance


def correlation(mu, rho, kappa, sigmasquare0, theta,
                    init_price, sim_n, step_n, T, K, r):
    """ get the correlation bwtween control variable and target variable
    sim_n: the number of simulations
    step_n: the number of steps
    """
    payoff = np.zeros([sim_n])
    STs = np.zeros([sim_n])
    std = (T / step_n) ** 0.5
    for i in range(0, sim_n):
        W = np.random.normal(0, std, step_n + 1)
        Z = np.random.normal(0, std, step_n + 1)
        ST = heston_single(mu, rho, kappa, sigmasquare0,
             theta, init_price, T, step_n, r, W, Z)
        STs[i] = ST
        payoff[i] = max(ST-K, 0)
    corre = corr_c(STs, payoff, init_price)
    return corre


def heston_control(mu, rho, kappa, sigmasquare0, theta,
                      init_price, sim_n, step_n, T, K, r):
    """
    Monte Carlo Milstein simulation with control variate
    sim_n: the number of simulations
    step_n: the number of steps
    """
    # estimate correlation first:
    corr = correlation(mu, rho, kappa, sigsquare0, theta,
                            init_price, sim_n, step_n, T, K, r)
    payoff = np.zeros([sim_n])
    std = (T / step_n) ** 0.5
    for i in range(0, sim_n):
        W = np.random.normal(0, std, step_n + 1)
        Z = np.random.normal(0, std, step_n + 1)
        ST = heston_single(mu, rho, kappa, sigmasquare0,
             theta, init_price, T, step_n, r, W, Z)
        payoff[i] = max(ST - K, 0) + corr*(ST - init_price)
    mean = np.mean(payoff)
    variance = np.var(payoff)
    return mean, variance


def heston_anti(mu, rho, kappa, sigmasquare0, theta,
                   init_price, sim_n, step_n, T, K, r):
    """
    Monte Carlo Milstein simulation with antithetic variate
    sim_n: the number of simulations
    step_n: the number of steps
    """
    # get correlation first:
    payoff = np.zeros([sim_n])
    std = (T / step_n) ** 0.5
    for i in range(0, sim_n):
        W = np.random.normal(0, std, step_n + 1)
        Z = np.random.normal(0, std, step_n + 1)
        ST1 = heston_single(mu, rho, kappa, sigmasquare0,
              theta, init_price, T, step_n, r, W, Z)
        payoff1 = max(ST1 - K, 0)
        # antithetic variate method:
        W1 = -W
        Z1 = -Z
        ST2 = heston_single(mu, rho, kappa, sigmasquare0,
              theta, init_price, T, step_n, r, W1, Z1)
        payoff2 = max(ST2 - K, 0)
        payoff[i] = (payoff1 + payoff2)/2
    mean = np.mean(payoff)
    variance = np.var(payoff)
    return mean, variance


def heston_condional_single(mu, rho, kappa, sigmasquare, theta, init_price,
                            T, step_n, K, W):
    """ single simulation of Heston model by conditional expectation
        Milstein scheme
    parameters:
        rho: correlation parameter
        sigmasquare: initial volatility
        init_price: initial price of asset
        T: time to maturity
        step_n: the steps of every simulation
        W: Brownian Motion
    return:
        the payoff of this simulation
    """
    r = 0  # assume interest is 0
    integral = 0
    integralstoch = 0
    delta_t = T / step_n
    for i in range(0, step_n):
        integral = integral + delta_t * sigmasquare
        integralstoch = integralstoch + (np.sqrt(sigmasquare)) * W[i]
        sigmasquare = sigmasquare + kappa*(theta-sigmasquare)*delta_t +\
                      mu*np.sqrt(sigmasquare)*W[i] +\
                      ((mu**2)/4)*(W[i]**2-delta_t)
        if sigmasquare < 0:
            sigmasquare = - sigmasquare
    S0_prem = init_price * np.exp(rho * integralstoch - 0.5 * integral * (rho ** 2))
    sig = np.sqrt((1 - (rho ** 2)) * integral / T)
    payoff = BlackScholes(S0_prem, K, r, sig, T)
    return payoff


def heston_conditional(mu, rho, kappa, sigmasquare0, theta, init_price,
                       sim_n, step_n, T, K):
    """ conditional Monte Carlo Milstein simulation
    sim_n: the number of simulations
    step_n: the number of steps
    """
    payoff = np.zeros([sim_n])
    std = (T / step_n) ** 0.5
    for i in range(0, sim_n):
        W = np.random.normal(0, std, step_n+1)
        payoff[i] = heston_condional_single(mu, rho, kappa, sigmasquare0,
                    theta, init_price, T, step_n, K, W)
    mean = np.mean(payoff)
    variance = np.var(payoff)
    return mean, variance


def heston_conditional_anti(mu, rho, kappa, sigmasquare0, theta, init_price,
                       sim_n, step_n, T, K):
    """ conditional Monte Carlo Milstein simulation with antithetic variate
    sim_n: the number of simulations
    step_n: the number of steps
    """
    payoff = np.zeros([sim_n])
    std = (T / step_n) ** 0.5
    for i in range(0, sim_n):
        W = np.random.normal(0, std, step_n+1)
        payoff1 = heston_condional_single(mu, rho, kappa, sigmasquare0,
                    theta, init_price, T, step_n, K, W)
        W1 = -W
        payoff2 = heston_condional_single(mu, rho, kappa, sigmasquare0,
                    theta, init_price, T, step_n, K, W1)
        payoff[i] = (payoff1 + payoff2)/2
    mean = np.mean(payoff)
    variance = np.var(payoff)
    return mean, variance

# Parematrs 1 and 1000 simulations

In [7]:
mu = 0.3
rho = -0.5
kappa = 1
sigsquare0 = 0.04
theta = 0.09
T = 1
K = 100
init_price = 100
r = 0
step_n = 1000
num = 50

In [22]:
sim_n = 500
payoffs_heston = np.zeros([num])
for i in tqdm(range(num)):
    mean_i, var_i = heston(mu, rho, kappa, sigsquare0, 
                           theta, init_price, sim_n, step_n, T, K, r)
    payoffs_heston[i] = mean_i
print('Milstein, mean: {}, std {}'.format(np.mean(payoffs_heston), np.std(payoffs_heston)))

100%|██████████| 50/50 [03:38<00:00,  4.37s/it]

Milstein, mean: 9.215033979431375, std 0.6639176393601411


In [23]:
sim_n = 250
payoffs_heston_anti = np.zeros([num])
for i in tqdm(range(num)):
    mean_i, var_i = heston_anti(mu, rho, kappa, sigsquare0, 
                           theta, init_price, sim_n, step_n, T, K, r)
    payoffs_heston_anti[i] = mean_i
print('Milstein, mean: {}, std {}'.format(np.mean(payoffs_heston_anti), np.std(payoffs_heston_anti)))

100%|██████████| 50/50 [03:47<00:00,  4.55s/it]

Milstein, mean: 9.162145545435594, std 0.4759094697884108


In [16]:
sim_n = 1000
payoffs_heston_control = np.zeros([num])
for i in tqdm(range(num)):
    mean_i, var_i = heston_control(mu, rho, kappa, sigsquare0, 
                           theta, init_price, sim_n, step_n, T, K, r)
    payoffs_heston_control[i] = mean_i
print('Milstein, mean: {}, std {}'.format(np.mean(payoffs_heston_control),
      np.std(payoffs_heston_control)))

100%|██████████| 50/50 [15:31<00:00, 18.64s/it]

Milstein, mean: 9.198698460078234, std 0.22440231421240533


In [10]:
sim_n = 1000
payoffs_cond = np.zeros([num])
for i in tqdm(range(num)):
    mean_i, var_i = heston_conditional(mu, rho, kappa, sigsquare0, theta, init_price,
                              sim_n, step_n, T, K)
    payoffs_cond[i] = mean_i
print('Milstein, mean: {}, std {}'.format(np.mean(payoffs_cond), np.std(payoffs_cond)))

100%|██████████| 50/50 [04:45<00:00,  5.71s/it]

Milstein, mean: 9.24888440546404, std 0.13023066060628558


In [15]:
sim_n = 500
payoffs_cond_anti = np.zeros([num])
for i in tqdm(range(num)):
    mean_i, var_i = heston_conditional_anti(mu, rho, kappa, sigsquare0, theta, init_price,
                              sim_n, step_n, T, K)
    payoffs_cond_anti[i] = mean_i
print('Milstein, mean: {}, std {}'.format(np.mean(payoffs_cond_anti), np.std(payoffs_cond_anti)))

100%|██████████| 50/50 [05:07<00:00,  6.14s/it]

Milstein, mean: 9.220427359267799, std 0.023135583677435068


In [34]:
a = ['a', 1.001, 2,989,3.786]

In [35]:
np.array(a)

array(['a', '1.001', '2', '989', '3.786'], dtype='<U5')

In [42]:
t1 = time.time()
for i in range(100000000):
    i += i
t2 = time.time()
t2-t1

16.53956413269043

In [43]:
import pandas as pd
df1 = pd.read_csv('./exp_res.csv')

In [46]:
df1[df1.columns[1:]]

,method name,call price mean,standard deviation,average time,simulation times,mu,rho,kappa,sigsquare0,theta
0,Normal Milstein,6.0023,6.6180,0.02,2,0.3,-0.4,0.5,0.02,0.05
1,Normal Milstein + antithetic variate,6.0663,4.2215,0.02,1,0.3,-0.4,0.5,0.02,0.05
2,Normal Milstein + control variate,5.5683,4.7820,0.02,2,0.3,-0.4,0.5,0.02,0.05
3,Normal Milstein + conditional expectation,5.9273,1.0590,0.01,2,0.3,-0.4,0.5,0.02,0.05
4,Normal Milstein + conditional expectation + an...,6.1114,0.4221,0.01,1,0.3,-0.4,0.5,0.02,0.05
5,Normal Milstein,5.4545,5.0455,0.04,4,0.3,-0.4,0.5,0.02,0.05
6,Normal Milstein + antithetic variate,5.8690,3.6282,0.04,2,0.3,-0.4,0.5,0.02,0.05
7,Normal Milstein + control variate,6.2375,3.6812,0.04,4,0.3,-0.4,0.5,0.02,0.05
8,Normal Milstein + conditional expectation,5.8954,0.8433,0.02,4,0.3,-0.4,0.5,0.02,0.05
9,Normal Milstein + conditional expectation + an...,5.8918,0.3726,0.02,2,0.3,-0.4,0.5,0.02,0.05
